# Importações e funções

Aqui encontram-se todas as importações e funções usadas para enriquecer o dataset "PhishingDataset.csv"

### Funções:

- <strong>get_ip(domain)</strong>: Esta função é utilizada para obter o endereço IP de um domínio (por exemplo, www.exemplo.com)
    - <strong>Funcionamento</strong>:
        - <strong>Bloco try</strong>:
            - Dentro do bloco try, a função tenta obter o endereço IP do domínio utilizando socket.gethostbyname(domain).
            - O resultado é impresso na tela usando print.
            - Em seguida, o endereço IP é retornado pela função.
        - <strong>Bloco except</strong>:
            - Se ocorrer qualquer exceção durante a execução do código dentro do bloco try, como por exemplo se o domínio não existir ou não for possível obter o IP, a exceção é capturada.
            - A função retorna None para indicar que não foi possível obter o endereço IP.


- <strong>get_location(ip)</strong>: Esta função tem como objetivo obter a localização geográfica e outras informações associadas a um endereço IP utilizando a API do serviço ipinfo.io.
    - <strong>Funcionamento</strong>:
        - Faz uma requisição HTTP GET à API ipinfo.io com o endereço IP fornecido e um token de acesso.
        - Converte a resposta JSON em um dicionário.
        - Extrai a latitude e longitude da localização.
        - Devolve um dicionário com informações como cidade, região, país, organização, código postal, fuso horário, nome do anfitrião, latitude e longitude.
    - <strong>Tratamento de Erros</strong>: Se ocorrer algum erro durante a requisição ou processamento dos dados, retorna um dicionário com campos vazios.
    

- <strong>get_dns_count(ip)</strong>: Esta função tem como objetivo obter o nome de domínio associado a um endereço IP (DNS reverso).
    - <strong>Funcionamento</strong>:
        - Utiliza a função gethostbyaddr do módulo socket para obter o DNS reverso do IP fornecido.
        - Devolve o nome de domínio associado.
    - <strong>Tratamento de Erros</strong>: Se ocorrer algum erro durante a resolução do DNS, retorna None.



In [1]:
import pandas as pd
import socket
import requests
import dns.resolver
import time

# Função para obter o IP de um domínio
def get_ip(domain):
    try:
        print (socket.gethostbyname(domain))
        return socket.gethostbyname(domain)
        
    except Exception as e:
        return None

# Função para obter a localização do IP usando a API ipinfo.io
def get_location(ip):
    if ip:
        try:
            response = requests.get(f'https://ipinfo.io/{ip}/json?token=b90bb6976ece63')
            data = response.json()
            print("Aqui",data)
            loc = data.get('loc', '').split(',')
            latitude = loc[0] if len(loc) > 0 else ''
            longitude = loc[1] if len(loc) > 1 else ''
            return {
                'city': data.get('city', ''),
                'region': data.get('region', ''),
                'country': data.get('country', ''),
                'org': data.get('org', ''),
                'postal': data.get('postal', ''),
                'timezone': data.get('timezone', ''),
                'hostname': data.get('hostname', ''),
                'latitude': latitude,
                'longitude': longitude,
            }
        except Exception as e:
            return {
                'city': '',
                'region': '',
                'country': '',
                'org': '',
                'postal': '',
                'timezone': '',
                'hostname': '',
                'latitude': '',
                'longitude': '',
            }
    return {
        'city': '',
        'region': '',
        'country': '',
        'org': '',
        'postal': '',
        'timezone': '',
        'hostname': '',
        'latitude': '',
        'longitude': '',
    }

# Função para obter o DNS reverso de um IP
def get_reverse_dns(ip):
    if ip:
        try:
            return socket.gethostbyaddr(ip)[0]
        except Exception as e:
            return None
    return None

# Função para obter o número de registros DNS de um IP
def get_dns_count(ip):
    if ip:
        try:
            answers = dns.resolver.resolve(ip, 'A')
            return len(answers)
        except Exception as e:
            return 0
    return 0


# Ler dataset
Leitura do dataset

In [2]:
'''
Já corridos: 
    -dataset1_block6

A correr:
    -teste
'''
# Carregar o dataset
df = pd.read_csv('Datasets/teste.csv', delimiter=';')


# Linhas em que a label não se encontra preenchida
Antes de enriquecermos o nosso dataset verificamos se existiam registos em que a nossa variável dependente "label" não possuía registos e se estes existiam, foram posteriormente removidos

In [3]:
# Substituir valores vazios na coluna "label" por -1
df['label'].fillna(-1, inplace=True)

# Contar os valores únicos na coluna "label"
label_counts = df['label'].value_counts()
print(label_counts)

# Encontrar os índices das linhas com valor -1 na coluna "label"
indices_to_drop = df[df['label'] == -1].index

# Remover as linhas com valor -1 na coluna "label"
df.drop(indices_to_drop, inplace=True)

label
 1.0    1113
 0.0     879
-1.0       8
Name: count, dtype: int64


C:\Users\PC\AppData\Local\Temp\ipykernel_25924\2910599684.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['label'].fillna(-1, inplace=True)


# Reverse module DNS
Este é o reverse module DNS, que será usado para enriquecer os nossos dados do nosso dataset.
Executa em blocos de 100 até ao fim do ficheiro para evitar que o endereço de IP do computador que estiver a utilizar seja bloqueado.
No final da execução, o mesmo é guardado num novo ficheiro chamado "total.csv"

In [4]:
# Adicionar contador para pausar a cada 100 registros
# Adicionar colunas vazias ao DataFrame
df['IP'] = ''
df['DNS'] = ''
df['Reverse DNS'] = ''
df['n_dns'] = ''
df['City'] = ''
df['Region'] = ''
df['Country'] = ''
df['Org'] = ''
df['Postal'] = ''
df['Timezone'] = ''
df['Hostname'] = ''
df['Latitude'] = ''
df['Longitude'] = ''

# Adicionar contador para pausar a cada 100 registros
for i in range(0, len(df), 100):
    # Processar blocos de 100 registros
    chunk = df.iloc[i:i+100].copy()

    # Adicionar colunas para armazenar IP, DNS, Reverse DNS e número de DNS
    chunk['IP'] = chunk['URL'].apply(lambda url: get_ip(url.split('//')[1]))
    chunk['DNS'] = chunk['URL'].apply(lambda url: url.split('//')[1])
    chunk['Reverse DNS'] = chunk['IP'].apply(get_reverse_dns)
    chunk['n_dns'] = chunk['IP'].apply(get_dns_count)

    # Obter informações de localização e adicionar colunas separadas
    location_data = chunk['IP'].apply(get_location)
    chunk['City'] = location_data.apply(lambda loc: loc['city'])
    chunk['Region'] = location_data.apply(lambda loc: loc['region'])
    chunk['Country'] = location_data.apply(lambda loc: loc['country'])
    chunk['Org'] = location_data.apply(lambda loc: loc['org'])
    chunk['Postal'] = location_data.apply(lambda loc: loc['postal'])
    chunk['Timezone'] = location_data.apply(lambda loc: loc['timezone'])
    chunk['Hostname'] = location_data.apply(lambda loc: loc['hostname'])
    chunk['Latitude'] = location_data.apply(lambda loc: loc['latitude'])
    chunk['Longitude'] = location_data.apply(lambda loc: loc['longitude'])

    # Juntar o chunk ao DataFrame original
    df.iloc[i:i+100] = chunk.copy()

    print(f"Processed chunk {i} to {i + 100}")

    if i > 0 and i % 100 == 0:
        print(f"Processed {i} records. Sleeping for 60 seconds...")
        time.sleep(60)

# Remover colunas não nomeadas
df.drop(df.columns[df.columns.str.contains('Unnamed', case=False)], axis=1, inplace=True)

# Salvar o resultado em um novo arquivo CSV
df.to_csv('teste_reverse_dns.csv', index=False, sep=';')

# Exibir o DataFrame
print(df.head())

52.178.158.175
209.141.46.186
172.67.143.104
185.176.43.98
34.88.4.51
3.64.163.50
87.233.198.111
192.95.16.17
159.69.67.170
3.160.132.82
23.194.139.23
85.158.182.48
67.214.133.18
107.154.141.50
156.234.234.198
3.160.132.75
23.227.38.74
104.21.235.207
92.205.229.154
63.141.128.20
135.181.92.226
52.17.119.105
153.92.0.100
89.185.234.13
23.227.38.74
142.251.37.243
82.71.205.12
172.67.218.16
172.67.202.6
194.191.24.44
104.21.30.43
208.215.218.15
68.66.192.169
23.185.0.1
169.1.20.252
152.91.31.24
104.21.6.227
23.227.38.74
135.84.124.41
160.153.138.94
103.27.34.24
172.67.72.119
104.21.63.23
104.26.8.177
85.255.199.141
64.190.63.222
207.58.136.71
44.221.239.236
40.118.100.127
13.86.112.87
151.101.1.124
199.242.239.182
46.8.8.100
146.75.90.132
54.209.77.18
212.224.112.72
52.224.186.118
3.160.132.78
64.62.171.34
172.67.189.85
212.78.94.3
23.227.38.74
35.208.70.82
163.171.138.115
188.172.241.251
192.0.66.104
172.67.138.162
Aqui {'ip': '52.178.158.175', 'city': 'Dublin', 'region': 'Leinster', 'co

# Código que não é usado
Estes blocos de código foram utilizados inicialmente para ajudar a formular o reverse e a dividir o dataset em blocos, mas não são necessários caso seja necessária aplicar de novo ao dataset

In [5]:
""" import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar o dataset
df = pd.read_csv('PhishingDataset.csv', delimiter='')

# Converter todas as colunas para strings
df = df.astype(str)

# Dividir o dataset em dois novos datasets (50% e 50%)
df1, df2 = train_test_split(df, test_size=0.5, random_state=42)

# Salvar os dois novos datasets em arquivos CSV separados
df1.to_csv('dataset1.csv', index=False,sep=';')
df2.to_csv('dataset2.csv', index=False,sep=';') """


" import pandas as pd\nfrom sklearn.model_selection import train_test_split\n\n# Carregar o dataset\ndf = pd.read_csv('PhishingDataset.csv', delimiter='')\n\n# Converter todas as colunas para strings\ndf = df.astype(str)\n\n# Dividir o dataset em dois novos datasets (50% e 50%)\ndf1, df2 = train_test_split(df, test_size=0.5, random_state=42)\n\n# Salvar os dois novos datasets em arquivos CSV separados\ndf1.to_csv('dataset1.csv', index=False,sep=';')\ndf2.to_csv('dataset2.csv', index=False,sep=';') "

In [6]:
""" import pandas as pd

# Carregar o dataset com delimitador ','
df = pd.read_csv('Datasets/total.csv', delimiter=',')

# Salvar o dataset com delimitador ';'
df.to_csv('Datasets/total.csv', index=False, sep=';')  """

""" import pandas as pd

# Carregar o dataset com delimitador ';'
df = pd.read_csv('Datasets/dataset1.csv', delimiter=';')

# Calcular o número de linhas em 20% do dataset
n = len(df)
block_size = int(n * 0.2)

# Shuffle the dataset
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Create a list to hold each 20% block
blocks = []

# Generate the 20% blocks
for i in range(5):
    start_index = i * block_size
    end_index = (i + 1) * block_size
    block = df_shuffled.iloc[start_index:end_index]
    blocks.append(block)
    # Save each block to a new CSV file
    block.to_csv(f'Datasets/dataset_block_{i+1}.csv', index=False, sep=';')

# In case the dataset size is not perfectly divisible by 5, add the remaining rows to the last block
if end_index < n:
    remaining_block = df_shuffled.iloc[end_index:]
    remaining_block.to_csv(f'Datasets/dataset_block_6.csv', index=False, sep=';')  """


" import pandas as pd\n\n# Carregar o dataset com delimitador ';'\ndf = pd.read_csv('Datasets/dataset1.csv', delimiter=';')\n\n# Calcular o número de linhas em 20% do dataset\nn = len(df)\nblock_size = int(n * 0.2)\n\n# Shuffle the dataset\ndf_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)\n\n# Create a list to hold each 20% block\nblocks = []\n\n# Generate the 20% blocks\nfor i in range(5):\n    start_index = i * block_size\n    end_index = (i + 1) * block_size\n    block = df_shuffled.iloc[start_index:end_index]\n    blocks.append(block)\n    # Save each block to a new CSV file\n    block.to_csv(f'Datasets/dataset_block_{i+1}.csv', index=False, sep=';')\n\n# In case the dataset size is not perfectly divisible by 5, add the remaining rows to the last block\nif end_index < n:\n    remaining_block = df_shuffled.iloc[end_index:]\n    remaining_block.to_csv(f'Datasets/dataset_block_6.csv', index=False, sep=';')  "